In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json

json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'
json_path_onedrive = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\'

### 1. Extract Stimulation Parameters

In [ ]:
json_files_to_match = pd.read_excel(os.path.join(
    json_path_onedrive, 'results', 'Stim_pars',
    'Jsons_12mfu_StimPars_ToExtract.xlsx'
))

json_files_to_match

In [ ]:
saving_path = os.path.join(
    json_path_onedrive, 'results', 'Stim_pars'
)

for index, row in json_files_to_match.iterrows():
    
    json_12mfu_stimPars = os.path.join(
        json_path_project, 
        json_files_to_match.loc[index,'SubID'],
        json_files_to_match.loc[index,'Con_reason'],
        json_files_to_match.loc[index,'json_fileName']
    )

    with open(json_12mfu_stimPars) as file:
                # Load the JSON data
                data = json.load(file)

    ElectrodeType = str(json_files_to_match.loc[index,'ElectrodeType'])
    
    stim_pars_dict = extract_features.extract_StimPars(data, ElectrodeType)

    subID = json_files_to_match.loc[index,'SubID']
    saving_json_name = f'{subID}_StimPars_12mfu.json'


    with open(os.path.join(
                saving_path,
                saving_json_name
            ), 'w') as file:
                json.dump(stim_pars_dict, file)

### 2. Calculate TEED

In [ ]:
stims_path = os.path.join(
    json_path_onedrive, 'results', 'Stim_pars'
)

with open(stims_path, 'r') as file:
    stims_dat = json.load(os.path.join(
        stims_path,
        'Sub005_StimPars_12mfu.json'
    ))


In [ ]:
json_path_onedrive